In [ ]:
import gtools
import pysam
from collections import defaultdict
from tqdm import tqdm
import numpy as np
import argparse
import sys
import multiprocessing
from os.path import exists
import os
import time
import pickle

In [ ]:
parser = argparse.ArgumentParser(description='''
Calculates the callable fragment coverage for duplex-seq. Takes a BAM file of duplex-seq reads as input and determines the number of callable fragments 
overlapping each position in the genome, in other words, the number of opportunities a mutation could have been detected and pass all duplex-seq filters 
at each position. This script is not 100% accurate, as it does not consider indels in reads, and indels alter the reference bases a read spans.
    ''')
parser.add_argument(dest='input', metavar='INPUT_FILE', type=str, 
                   help='BAM file to call from. Must be coordinate sorted and have an index file')
parser.add_argument(dest='output', metavar='OUTPUT_PREFIX', type=str,
                   help='prefix to append to output numpy arrays, final outputs will be {prefix}{chrom}.npy for each chromosome')
# parser.add_argument('-m', '--min_mapped_cov', dest='min_mapped_cov', metavar='MIN_MAPPED_COV', type=int, default=1,
#                    help='number of reads from each mapped strand required to call variants (default: 1)')
parser.add_argument('-t', '--min_total_cov', dest='min_total_cov', metavar='INT', type=int, default=6,
                   help='number of covering reads required to call variants within a fragment (default: 6)')
parser.add_argument('-s', '--min_orig_cov', dest='min_orig_cov', metavar='INT', type=int, default=2,
                   help='number of reads required from each original strand to call variants within a fragment (default: 2)')
parser.add_argument('-q', '--min_mq', dest='min_mq', metavar='FLOAT', type=int, default=10,
                   help='minimum average mq of pcr duplicate reads required to call variants within a fragment (default: 10)')
parser.add_argument('-@', '--threads', dest='threads', metavar='INT', type=int, default=1,
                   help='chromosomes are divided between threads. Cannot use more threads than num chromosomes + 1 (default: 1)')
parser.add_argument('-b', '--blacklist', dest='blacklist', metavar='PREFIX', type=str, default=None,
                   help='''prefix of the blacklist files generated by duplex_blacklist_regions. Files should be 
                   "{prefix}{contig}_{type}.npy" where type is coverage_percentile, duplicate_distance, poly_at_length, 
                   and poly_repeat_length (default: do not use blacklist)''')
parser.add_argument('-c', '--blacklist_cutoffs', dest='blacklist_cutoffs', metavar='FLOAT,INT,INT,INT', type=str, default='0.007,1,-1,7,4',
                   help='''cutoffs to use for the blacklist. Same order as listed in "-b" help section. Values equal to the cutoff are 
                    counted as callable (default: 0.007,1,-1,7,4)''')
parser.add_argument('-e', '--end_dist', dest='end_dist', metavar='INT', type=int, default=5,
                   help='minimum distance from the fragment end required to call a mutation. The end mismatch rate filter makes \
                   fragment ends uncallable (default: 5)')
parser.add_argument('-l', '--max_frag_len', dest='max_frag_len', metavar='INT', type=int, default=300,
                   help='maximum fragment length before a fragment is uncallable (default: 300)')
parser.add_argument('-r', '--require_overlap', dest='require_overlap', action='store_true',
                   help='whether read1-read2 overlap is required to call a variant. WARNING: setting this only gives the correct \
                   output when reads are 150bp (default: not required)')
parser.add_argument('-u', '--umi', dest='umi', metavar='UMI', type=str, default=None,
                   help='Consider the UMI in the provided BAM tag when grouping pre-PCR fragments (default: no UMI)')

try: # run this if in a jupyter notebook
    get_ipython()
    print('Determined code is running in Jupyter')
    if os.getcwd()[:8] != '/scratch': # switch to the scratch directory where all the data files are
        os.chdir(f'/scratch/cam02551/{os.getcwd().split("/")[-2]}')
    args = parser.parse_args('data/align/chandler_freq_problem.bam tmp/coverage_test_ --max_frag_len 2000 -@ 1 -b data/blacklist/arabidopsis/duplex_blacklist_ --umi RX'.split()) # used for testing
except: # run this if in a terminal
    args = parser.parse_args()

sys.stderr.write('Running duplex_coverage.py with arguments:\n' + '\n'.join([f'{key}={val}' for key, val in vars(args).items()]) + '\n')

if not exists(args.input):
    sys.stderr.write('ERROR: input file {} does not exist\n'.format(args.input))
    exit()

aln = pysam.AlignmentFile(args.input, 'rb') # load BAM for doing checks

# assert index can be opened
try:
    aln.check_index()
except:
    sys.stderr.write('ERROR: failed to open index file {}\n'.format(args.input + '.bai'))
    exit()

if args.output and '/' in args.output:
    os.makedirs(os.path.dirname(args.output), exist_ok=True)

In [ ]:
# confirm that file is coordinate sorted
last = ('', 0)
read_length = -1
for i, read in enumerate(aln.fetch()):
    # check if current read aligned before previous read (out of order)
    nex = (read.reference_name, read.reference_start)
    if nex < last:
        sys.stderr.write('ERROR: BAM file is not coordinate sorted\n')
        exit()
    last = nex
    read_length = max(read_length, read.query_length)
    if i > 10000:
        break
sys.stderr.write(f'inferred a read length of {read_length}bp\n')

In [ ]:
chroms = [x.contig for x in aln.get_index_statistics()]
chroms.sort()

In [ ]:
if args.blacklist is not None:
    # all are indexed as bl[chrom][pos] = value
    bl_cov_per = dict() # coverage percentile
    bl_dup_dist = dict() # mismatches required to have a duplicate 120mer in the genome
    bl_poly_at = dict() # length of poly-A or poly-T starting or ending at position
    bl_poly_rep = dict() # length of di or trinucleotide repeat starting or ending at position
    sys.stderr.write('Loading blacklist files\n')
    for chrom in tqdm(chroms):
        bl_cov_per[chrom] = np.load(f'{args.blacklist}{chrom}_coverage_percentile.npy')
        bl_dup_dist[chrom] = np.load(f'{args.blacklist}{chrom}_duplicate_distance.npy')
        bl_poly_at[chrom] = np.load(f'{args.blacklist}{chrom}_poly_at_length.npy')
        bl_poly_rep[chrom] = np.load(f'{args.blacklist}{chrom}_poly_repeat_length.npy')
    
    # extract the blacklist cutoffs from the command line parameter
    s = args.blacklist_cutoffs.split(',')
    assert len(s) == 5
    cutoff_cov_min = float(s[0])
    cutoff_cov_max = float(s[1])
    cutoff_dup_dist = int(s[2])
    cutoff_poly_at = int(s[3])
    cutoff_poly_rep = int(s[4])
    
    # combine the cov_per, poly_at, and poly_rep blacklists to a single boolean blacklist where true means callable
    bl_bool = dict()
    for chrom in chroms:
        bl_bool[chrom] = ~((bl_cov_per[chrom] < cutoff_cov_min) | (bl_cov_per[chrom] > cutoff_cov_max)) & \
                         (bl_poly_at[chrom] <= cutoff_poly_at) & (bl_poly_rep[chrom] <= cutoff_poly_rep)
    del bl_cov_per, bl_poly_at, bl_poly_rep
else:
    sys.stderr.write('No blacklist files provided, defaulting to whitelisting all of genome\n')
    cutoff_dup_dist = -1
    
    bl_bool = dict()
    bl_dup_dist = dict()
    for chrom in chroms:
        bl_bool[chrom] = np.full(aln.get_reference_length(chrom), True)
        bl_dup_dist[chrom] = np.zeros(aln.get_reference_length(chrom))

In [ ]:
read_counts = {x.contig:x.total for x in aln.get_index_statistics()}

In [ ]:
def chrom_coverage(file, chrom):
    frags = defaultdict(lambda: [0, 0, 0]) # key (frag_start, tlen, umi), value [+ strand read pairs, - strand read pairs, total MQ of forward reads]
    aln = pysam.AlignmentFile(args.input, 'rb')
    output = np.zeros(aln.get_reference_length(chrom), dtype=np.intc)
    i = 0
    for read in tqdm(aln.fetch(contig=chrom), position=0, mininterval=10, total=read_counts[chrom], desc=chrom):
        i += 1
        # periodically flush
        if i % 100000 == 0 or i == read_counts[chrom]:
            to_flush = []
            for frag in frags:
                if i != read_counts[chrom] and read.reference_start <= frag[0]: # if we haven't passed all the reads for the current fragment
                    break
                to_flush.append(frag)
            for frag in to_flush:
                v = frags[frag]
                del frags[frag]
                
                if v[2] / (v[0] + v[1]) < args.min_mq: # insufficient MQ
                    continue
                if bl_dup_dist[chrom][frag[0]] < cutoff_dup_dist: # fragment fails the dup_dist blacklist
                    continue
                if frag[1] > args.max_frag_len: # fragment is too large
                    continue
                
                overlap_callable = (v[0] * 2 >= args.min_orig_cov) and (v[1] * 2 >= args.min_orig_cov) and ((v[0] + v[1]) * 2 >= args.min_total_cov)
                all_callable = (v[0] >= args.min_orig_cov) and (v[1] >= args.min_orig_cov) and ((v[0] + v[1]) >= args.min_total_cov) and not args.require_overlap

                if all_callable:
                    # note: this is safe even if the slice end is less than the start, arr[5:3] += 1 does nothing
                    output[frag[0] + args.end_dist:min(frag[0] + read_length, frag[0] + frag[1] - args.end_dist)] += 1 # add coverage for the forward read
                    output[max(frag[0] + args.end_dist, frag[0] + frag[1] - read_length):frag[0] + frag[1] - args.end_dist] += 1 # add coverage for the reverse read
                    output[max(frag[0] + args.end_dist, frag[0] + frag[1] - read_length):min(frag[0] + read_length, frag[0] + frag[1] - args.end_dist)] -= 1 # subtract double-counted coverage in the overlap
                elif overlap_callable:
                    output[max(frag[0] + args.end_dist, frag[0] + frag[1] - read_length):min(frag[0] + read_length, frag[0] + frag[1] - args.end_dist)] += 1
                    
        # thow out read pairs that don't align as expected, ignore reverse reads so we don't double count each fragment
        if read.is_unmapped or read.mate_is_unmapped or not (read.flag & 2) or read.is_reverse:
            continue
        
        # find the fragment start and end. Used to find PCR duplicates
        frag_start = min(read.reference_start, read.next_reference_start)
        tlen = abs(read.template_length)
        frag_strand = '+' if read.is_read1 else '-'
        umi = '' if args.umi is None else read.get_tag(args.umi)
            
        if frag_strand == '+':
            frags[(frag_start, tlen, umi)][0] += 1
        else:
            frags[(frag_start, tlen, umi)][1] += 1
        frags[(frag_start, tlen, umi)][2] += read.mapping_quality
    
    output *= bl_bool[chrom] # only count positions that pass the blacklists
    
    sys.stderr.write(f'made coverage array for {chrom}\n')
    
    np.save(f'{args.output}{chrom}.npy', output)

    sys.stderr.write(f'wrote coverage array for {chrom}\n')

In [ ]:
if __name__ == '__main__':
    # start worker processes from pool
    pool = multiprocessing.Pool(processes=max(1, args.threads - 1))
    processes = []
    for chrom in chroms: # for each chromosome, add a process
        processes.append(pool.apply_async(chrom_coverage, (args.input, chrom)))
    pool.close()
    pool.join()

    # check for errors
    for p in processes:
        p.get()

In [ ]:
args.max_frag_len

In [ ]:
sys.stderr.write(f'completed duplex_coverage.py on {args.input}\n')

## Check output

In [ ]:
# coverages = dict()
# for chrom in chroms:
#     with open(f'{args.output}{chrom}.pkl', 'rb') as f:
#         coverages[chrom] = pickle.load(f)

In [ ]:
# coverages.keys()

In [ ]:
# for i in range(1000):
#     print(i, coverages['Chr1'][i])